In [21]:
import numpy as np
import pandas as pd
from utils import BSplineFeatures

# Question 3

## a) Generate a vector x consisting of 50 points drawn at random from Uniform[0,1]

In [22]:
X = np.random.uniform(0,1,50)
# save simulation result in a dict
Y_dict = {}

## b) Generate 100 training sets.

In [23]:
X = np.random.uniform(0,1,50)
y = (np.sin(2*np.pi*X))**3 + np.random.normal(0,1,50)
X = X.reshape(-1,1)

### i. Fit the data with methods/models listed below.

OLS with linear model

In [24]:
from sklearn.linear_model import LinearRegression
y_list = []
for _ in range(0, 100):
    X = np.random.uniform(0,1,50)
    y = (np.sin(2*np.pi*X))**3 + np.random.normal(0,1,50)
    X = X.reshape(-1,1)
    reg = LinearRegression().fit(X, y)
    y_ols = reg.predict(X).reshape(-1,1)
    y_list.append(y_ols)

# print('Coefficients: \n', reg.coef_)
# print('Intercept: \n', reg.intercept_)
Y_ols = np.concatenate(y_list, axis=1)
Y_dict['ols'] = Y_ols

OLS with cubic polynomial model

In [25]:
from sklearn.preprocessing import PolynomialFeatures
y_list = []
for _ in range(0, 100):
    X = np.random.uniform(0,1,50)
    y = (np.sin(2*np.pi*X))**3 + np.random.normal(0,1,50)
    X = X.reshape(-1,1)
    poly = PolynomialFeatures(degree=3)
    X_ = poly.fit_transform(X)
    X_ = X_[:,1:]
    reg = LinearRegression().fit(X_, y)
    y_polyols = reg.predict(X_).reshape(-1,1)
    y_list.append(y_polyols)

# print('Coefficients: \n', reg.coef_)
# print('Intercept: \n', reg.intercept_)
Y_poly = np.concatenate(y_list, axis=1)
Y_dict['poly'] = Y_poly

Cubic spline (or B-spline) with 2 knots at 0.33 and 0.66

In [26]:
from scipy.interpolate import BSpline, splrep, splev


y_list = []
for _ in range(0, 100):
    X = np.sort(np.random.uniform(0,1,50))
    y = (np.sin(2*np.pi*X))**3 + np.random.normal(0,1,50)
    Xck = splrep(X, y, t=[0.33, 0.66], k=3)
    y_cubic_spl = splev(X, Xck).reshape(-1,1)
    y_list.append(y_cubic_spl)

# print('Coefficients: \n', reg.coef_)
# print('Intercept: \n', reg.intercept_)
Y_cubic_spl = np.concatenate(y_list, axis=1)
Y_dict['cubic_spl'] = Y_cubic_spl


Natural cubic spline with 5 knots at 0.1,0.3,0.5,0.7,0.9

In [27]:

from scipy.interpolate import CubicSpline, interp1d

y_list = []
for _ in range(0, 100):
    X = np.sort(np.random.uniform(0,1,50))
    y = (np.sin(2*np.pi*X))**3 + np.random.normal(0,1,50)
    ns = CubicSpline(X, y,bc_type='natural', extrapolate=True)
    y_cubic_nat_spl = ns(X).reshape(-1,1)
    y_list.append(y_cubic_nat_spl)

# print('Coefficients: \n', reg.coef_)
# print('Intercept: \n', reg.intercept_)
Y_cubic_nat_spl = np.concatenate(y_list, axis=1)
Y_dict['cubic_nat_spl'] = Y_cubic_nat_spl

Smoothing spline with tuning parameter chosen by GCV

In [28]:
import numpy as np
import csaps

y_list = []
for _ in range(0, 100):
    X = np.sort(np.random.uniform(0,1,50))
    y = (np.sin(2*np.pi*X))**3 + np.random.normal(0,1,50)
    sp = csaps.UnivariateCubicSmoothingSpline(X, y)
    y_smooth_spl = sp(X).reshape(-1,1)
    y_list.append(y_smooth_spl)

Y_smooth_spl = np.concatenate(y_list, axis=1)
Y_dict['smooth_spl'] = Y_smooth_spl


### ii. Compute the vector of fitted value y_pred obtained from each method/model.

In [ ]:
Y_dict

## c) for each method/model, you obtain a matrix of fitted values

## d) For each method/model, compute the pointwise variance of fitted values across the 100 training sets. Plot the pointwise variance curves (against x) for each method/model.

# Question 4

The South African heart disease data. Divide the dataset into a training set consisting of the first 300 observations, and
a test set consisting of the remaining observations. Apply logistic regression,
LDA and QDA on the training set. For each method, report the test error and
its standard error over the test set. Briefly discuss your results.

In [29]:
df = pd.read_csv('data/africa_heart.csv')